In [1]:
import os
import shutil
import contextlib
import pathlib
import json

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import run
import environment
import evolve
import encode
import plot

In [2]:
run1_results_dir = 'results/nearest_sbx_2/'

run1_vision = environment.NearestFoodParticleVision()
run1_genome_size = [run1_vision.organism_input_shape, 4, 2]

fitness = evolve.FoodConsumnptionToDistanceFitness()

arguments = {
    'start_organism_number': 40,
    'width': 12,
    'height': 12,
    'iterations': 960,
    'generation_time': 80,
    'organism_size': 0.12,
    'food_size': 0.05,
    'organism_vision_range': 1,
    'results_dir': run1_results_dir,
    'vision': run1_vision,
    'food_energy': 2,
    'food_appearance_number_rate': 0.7,
    'encoding': encode.RealValued(),
    'selection': evolve.TruncationSelection(fitness=fitness, n=20),
    'crossover': evolve.SBXCrossover(n=2),
    'mutation': evolve.GaussianMutation(mu=0, sigma=0.1, p=0.2),
    'elitism': 20,
    'genome_size': run1_genome_size,
    'food_particles_at_start': 100,
    'remove_dead_organisms': False,
}

# run1_params_file = os.path.join(run1_results_dir, 'data.json')

In [3]:
if os.path.exists(run1_results_dir):
    ans = input('Previous results folder exists. Do you want to delete it?')
    if ans == 'y':
        shutil.rmtree(run1_results_dir)

with contextlib.suppress(Exception):
    os.mkdir(run1_results_dir)
    
# with open(run1_params_file, 'w') as file:
#     json.dump(arguments, file)

run1 = run.OrganismsSimpleEnvironmentRunTool(**arguments)
run1.run()

Simulating organisms:   0%|          | 2/960 [00:00<03:51,  4.15it/s, Number of organisms=40, Gen=0]


KeyboardInterrupt: 

In [ ]:
org_loc = pq.read_table(os.path.join(run1_results_dir,
                                     'organisms_locations')).to_pandas()
org_stat = pq.read_table(os.path.join(run1_results_dir,
                                      'organisms_stats')).to_pandas()
food_loc = pq.read_table(os.path.join(run1_results_dir,
                                      'food_locations')).to_pandas()
food_stat = pq.read_table(os.path.join(run1_results_dir,
                                       'eaten_food')).to_pandas()


org_loc.reset_index(drop=True, inplace=True)
org_stat.reset_index(drop=True, inplace=True)
food_loc.reset_index(drop=True, inplace=True)
food_stat.reset_index(drop=True, inplace=True)

In [ ]:
org_loc.describe()

,index,x,y,theta,iteration,generation,v,a,energy,age
count,351500.000000,351500.000000,351500.000000,351500.000000,351500.000000,351500.000000,351500.000000,351500.000000,351500.000000,351500.000000
mean,1016.283073,7.930043,7.081728,-6.369097,4978.376245,49.288762,0.020765,0.011464,-5.839716,92.847084
std,580.122363,4.340035,4.553903,10.115935,2898.605735,28.984620,0.079832,0.047910,4.439766,59.292643
min,0.000000,0.000000,0.000000,-41.217814,0.000000,0.000000,-0.100000,-0.050000,-15.920000,0.000000
25%,513.000000,4.282505,3.120288,-10.818444,2467.000000,24.000000,-0.083333,-0.050000,-9.280000,43.000000
50%,1016.000000,7.975514,7.052269,-3.119308,4978.000000,49.000000,0.083333,0.050000,-5.360000,86.000000
75%,1519.000000,11.691109,10.904121,-0.581652,7489.000000,74.000000,0.087831,0.050000,-2.080000,141.000000
max,2029.000000,15.000000,15.000000,40.000000,9999.000000,99.000000,0.100000,0.050000,8.480000,399.000000


In [ ]:
org_stat.describe()

,id,age,iteration,energy
count,351500.000000,351500.000000,351500.000000,351500.000000
mean,1016.283073,92.847084,4978.376245,-5.839716
std,580.122363,59.292643,2898.605735,4.439766
min,0.000000,0.000000,0.000000,-15.920000
25%,513.000000,43.000000,2467.000000,-9.280000
50%,1016.000000,86.000000,4978.000000,-5.360000
75%,1519.000000,141.000000,7489.000000,-2.080000
max,2029.000000,399.000000,9999.000000,8.480000


In [ ]:
food_stat.describe()

,organism_id,food_location,energy_taken,iteration
count,6290.000000,6290.000000,6290.0,6290.000000
mean,1026.670906,7.492586,2.0,5012.527822
std,560.890311,4.320469,0.0,2805.681510
min,2.000000,0.004078,2.0,0.000000
25%,546.000000,3.753972,2.0,2594.000000
50%,1030.000000,7.498720,2.0,5008.000000
75%,1500.000000,11.187942,2.0,7367.000000
max,2029.000000,14.998826,2.0,9999.000000


In [ ]:
org_loc.groupby('generation')[['energy', 'age']].describe()

energy                                                       
             count      mean       std    min    25%   50%   75%   max   
generation                                                               
0           5000.0 -3.370400  2.311654  -8.00  -5.12 -3.28 -1.52  3.60  \
1           3500.0 -6.028000  3.996213 -14.00  -9.22 -5.84 -2.56  1.92   
2           3500.0 -6.466286  4.890646 -15.92 -10.64 -6.08 -2.40  6.16   
3           3500.0 -6.419429  4.739100 -15.92 -10.16 -6.16 -2.48  6.64   
4           3500.0 -6.850286  4.694363 -15.92 -10.88 -6.24 -2.64  1.20   
...            ...       ...       ...    ...    ...   ...   ...   ...   
95          3500.0 -6.010286  4.592822 -15.92  -9.76 -5.28 -2.00  1.92   
96          3500.0 -5.722857  4.164103 -15.92  -8.80 -5.44 -2.32  5.20   
97          3500.0 -6.300000  4.329447 -15.92  -9.60 -5.92 -2.56  2.00   
98          3500.0 -5.749714  4.699051 -15.92  -9.60 -5.28 -1.60  4.16   
99          3500.0 -5.635429  4.272932 -15.92  -8.64 -5.12 -2.08  2.08   

               age                                                         
             count       mean        std  min    25%   50%     75%    max  
generation                                                                 
0           5000.0  50.500000  28.868957  1.0  25.75  50.5   75.25  100.0  
1           3500.0  92.785714  57.726997  0.0  43.00  87.0  142.00  200.0  
2           3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  
3           3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  
4           3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  
...            ...        ...        ...  ...    ...   ...     ...    ...  
95          3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  
96          3500.0  95.214286  62.349193  0.0  43.00  87.0  144.00  299.0  
97          3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  
98          3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  
99          3500.0  92.357143  57.298932  0.0  43.00  87.0  141.00  199.0  

[100 rows x 16 columns]

In [ ]:
# frames_dir = os.path.join(run1_results_dir, 'frames')
# frames_dir = pathlib.Path(run1_results_dir, 'frames')
# frames_dir.mkdir()
# plot.create_frames(org_loc, food_loc, arguments['width'], arguments['height'], arguments['organism_size'], arguments['food_size'], frames_dir)

100%|█████████▉| 9980/9999 [50:58<00:05,  3.26it/s]  


In [1]:
# plot.generate_video(
#     os.path.join(run1_results_dir, 'frames/'),
#     framerate=24,
#     output=f'{run1_results_dir}evolution.mp4',
# )

CalledProcessError: Command 'b"\nplot.generate_video(\n    os.path.join(run1_results_dir, 'frames/'),\n    framerate=24,\n    output=f'{run1_results_dir}evolution.mp4',\n)\n"' returned non-zero exit status 1.

# Inspecting results

In [ ]:
from plotly import express as px
import plotly.graph_objs as go

In [ ]:
WINDOW = 5

moving_age_mean = org_loc[[
    'iteration', 'age'
]].groupby('iteration').mean().rolling(WINDOW).mean()

moving_age_max = org_loc[['iteration', 'age'
                          ]].groupby('iteration').max().rolling(WINDOW).mean()

moving_energy_mean = org_loc[[
    'iteration', 'energy'
]].groupby('iteration').mean().rolling(WINDOW).mean()

moving_energy_max = org_loc[[
    'iteration', 'energy'
]].groupby('iteration').max().rolling(WINDOW).mean()

x = np.arange(len(moving_age_max))

ma_age = go.Scatter(
    x=x,
    y=moving_age_mean['age'],
    name=f'Moving Age Average {WINDOW}',
    mode='lines',
    legendgroup='Age',
)
mm_age = go.Scatter(x=x,
                    y=moving_age_max['age'],
                    name=f'Moving Age Max {WINDOW}',
                    mode='lines',
                    legendgroup='Age')
ma_energy = go.Scatter(x=x,
                       y=moving_energy_mean['energy'],
                       name=f'Moving Energy Average {WINDOW}',
                       mode='lines',
                       legendgroup='Energy')
mm_energy = go.Scatter(x=x,
                       y=moving_energy_max['energy'],
                       name=f'Moving Energy Max {WINDOW}',
                       mode='lines',
                       legendgroup='Energy')

layout = go.Layout(title='Evolution Statistics',
                   xaxis=dict(title='Time'))

fig = go.Figure(data=[ma_age, mm_age, mm_energy, ma_energy], layout=layout)
fig.show()

In [ ]:
WINDOW = 3

moving_age_mean = org_loc[[
    'generation', 'age'
]].groupby('generation').mean().rolling(WINDOW).mean()

moving_age_max = org_loc[['generation', 'age'
                          ]].groupby('generation').max().rolling(WINDOW).mean()

moving_energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean().rolling(WINDOW).mean()

moving_energy_max = org_loc[['generation', 'energy']].groupby('generation').max().rolling(WINDOW).mean()

x = np.arange(len(moving_age_max))

ma_age = go.Scatter(
    x=x,
    y=moving_age_mean['age'],
    name=f'Moving Age Average {WINDOW}',
    mode='lines',
    legendgroup='Age',
)

mm_age = go.Scatter(x=x,
                    y=moving_age_max['age'],
                    name=f'Moving Age Max {WINDOW=}',
                    mode='lines',
                    legendgroup='Age')

ma_energy = go.Scatter(x=x,
                    y=moving_energy_mean['energy'],
                    name=f'Moving Energy Max {WINDOW=}',
                    mode='lines',
                    legendgroup='Energy')

mm_energy = go.Scatter(x=x,
                    y=moving_energy_mean['energy'],
                    name=f'Moving Energy Average {WINDOW=}',
                    mode='lines',
                    legendgroup='Energy')

layout = go.Layout(title='Evolution Statistics',
                   xaxis=dict(title='Generation'))

fig = go.Figure(data=[ma_age, mm_age, mm_energy, ma_energy], layout=layout)
fig.show()

In [ ]:
age_mean = org_loc[[
    'generation', 'age'
]].groupby('generation').mean()
age_max = org_loc[['generation', 'age'
                          ]].groupby('generation').max()
energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean()

x = np.arange(len(age_max))

age_avg = go.Scatter(
    x=x,
    y=age_mean['age'],
    name=f'Age Average',
    mode='lines',
    legendgroup='Age',
)

age_max = go.Scatter(x=x,
                    y=age_max['age'],
                    name=f'Age Max',
                    mode='lines',
                    legendgroup='Age')

energy_mean = go.Scatter(x=x,
                    y=energy_mean['energy'],
                    name=f'Energy Average',
                    mode='lines',
                    legendgroup='Energy')

layout = go.Layout(title='Evolution Statistics',
                   xaxis=dict(title='Generation'))

fig = go.Figure(data=[energy_mean], layout=layout)
# fig = go.Figure(data=[age_avg, age_max, energy_mean], layout=layout)

fig.show()

In [ ]:
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])